# Rollup of Total Grant Value for all US Counties

Grant information by state was compiled by [Investigate West](http://www.invw.org/2012/06/11/lwcf-grants-database-1283/) for the period 1965-2011.

In [80]:
import glob
import re

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [48]:
# Collect all file names and process into DataFrames
state_files = glob.glob('./Data/StateGrantData/*.xlsx')
dfs = []

for file in state_files:
    dfs.append(pd.read_excel(file))

all_states = pd.concat(dfs, ignore_index=True)
all_states.head()

,Grant ID & Element,Grant Name,Sponsor,County,State,Grant Amount,Year Approved,Year Completed,Type
0,3 - XXX,INDIAN PEAKS - BAKER CANYON,STATE OF UTAH,BEAVER,UT,5985.92,1966,1966,Development
1,4 - XXX,WEST BEAR RIVER BAY,STATE OF UTAH,BOX ELDER,UT,18500.00,1966,1967,Development
2,6 - XXX,WELLSVILLE PARK,CITY OF WELLSVILLE,CACHE,UT,934.25,1966,1967,Acquisition
3,2 - XXX,FARMINGTON BAY WATERFOWL AREA,STATE OF UTAH,DAVIS,UT,15150.00,1966,1967,Acquisition
4,11 - XXX,4 CITY PARKS,CITY OF BOUNTIFUL,DAVIS,UT,16225.15,1966,1966,Development


In [49]:
all_states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41145 entries, 0 to 41144
Data columns (total 9 columns):
Grant ID & Element    41145 non-null object
Grant Name            41145 non-null object
Sponsor               41145 non-null object
County                41145 non-null object
State                 41145 non-null object
Grant Amount          41145 non-null float64
Year Approved         41145 non-null int64
Year Completed        41145 non-null int64
Type                  41145 non-null object
dtypes: float64(1), int64(2), object(6)
memory usage: 2.8+ MB


In [50]:
all_states['State'].sort_values().unique()

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

In [51]:
# Count states - should be 51 (includes DC)
all_states['State'].nunique()

51

In [52]:
# Make all counties lowercase
all_states['County'] = all_states['County'].map(lambda s: s.lower())

In [70]:
# Fix pre-group structural issues (different spellings)
all_states['County'] = all_states['County'].map(lambda s: s.replace('saint', 'st.'))
all_states['County'] = all_states['County'].map(lambda s: s.replace('(city)', 'city'))

## Group County Data

In [71]:
# Create grouped grant data
all_counties = all_states.groupby(['State', 'County'])['Grant Amount'].agg(['count', 'sum'])

# Reset multi-index to flatten columns
all_counties = all_counties.reset_index()

# Re-name columns
all_counties.columns = ['state', 'county_orig', 'num_grants', 'grant_value']

all_counties.head()

,state,county_orig,num_grants,grant_value
0,AK,aleutians east,5,123552.58
1,AK,aleutians west,1,52405.00
2,AK,anchorage,67,9475506.86
3,AK,bethel,3,366133.32
4,AK,dillingham,2,96119.80


In [72]:
# Drop multi-county, planning, and no county rows
drop_rows = ['multi-county', 'planning', 'no county', 'county name missing']
all_counties = all_counties[~all_counties['county_orig'].isin(drop_rows)]

In [84]:
all_counties[all_counties['state'] == 'RI']

,state,county_orig,num_grants,grant_value
2294,RI,bristol,17,2504765.02
2295,RI,kent,52,4462589.63
2297,RI,newport,35,4809097.79
2299,RI,providence,139,13877173.38
2300,RI,washington,75,12637117.77


In [74]:
# Fix one-off structural differences between county columns and id data

# AK
all_counties['county_orig'].replace('matanuska susitna', 'matanuska-susitna', inplace=True)
all_counties['county_orig'].replace('wales hyder', 'wales-hyder', inplace=True)
all_counties['county_orig'].replace('skagway hoonah angoon', 'skagway', inplace=True)
all_counties['county_orig'].replace('valdez cordova', 'valdez-cordova', inplace=True)
all_counties['county_orig'].replace('yukon koyukuk', 'yukon-koyukuk', inplace=True)

# MO
all_counties['county_orig'].replace('ste genevieve', 'ste. genevieve', inplace=True)

In [77]:
all_counties['county_orig'].count()

2964

In [98]:
# Create key column to merge on
all_counties['county_key'] = all_counties['county_orig'] + ' ' + all_counties['state']
all_counties.head()

,state,county_orig,num_grants,grant_value,county_key
0,AK,aleutians east,5,123552.58,aleutians east AK
1,AK,aleutians west,1,52405.00,aleutians west AK
2,AK,anchorage,67,9475506.86,anchorage AK
3,AK,bethel,3,366133.32,bethel AK
4,AK,dillingham,2,96119.80,dillingham AK


## Process County ID Data

In [78]:
# Import county id data
county_ids = pd.read_csv('./Data/county_names.csv', sep=',')
county_ids.head()

,id,state,county
0,1001,Alabama,Autauga County
1,1003,Alabama,Baldwin County
2,1005,Alabama,Barbour County
3,1007,Alabama,Bibb County
4,1009,Alabama,Blount County


In [79]:
county_ids['county'].count()

3219

In [87]:
# Create state name and abbrev dictionary
st_name = county_ids['state'].unique()
st_abbrev = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
             'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
             'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
             'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
             'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR']


state_dict = dict(zip(st_name, st_abbrev))

In [90]:
# Add state abbreviation column
county_ids['state_abbv'] = county_ids['state'].map(lambda s: state_dict[s])
county_ids.head()

,id,state,county,county_key,state_abbv
0,1001,Alabama,Autauga County,autauga,AL
1,1003,Alabama,Baldwin County,baldwin,AL
2,1005,Alabama,Barbour County,barbour,AL
3,1007,Alabama,Bibb County,bibb,AL
4,1009,Alabama,Blount County,blount,AL


In [96]:
# Edit county names to tie to grant data county names, create key column to merge on
county_ids['county_key'] = county_ids['county'].map(lambda s: s.lower())

pattern = re.compile(r' county| census area| municipality| borough| city and borough| parish')
county_ids['county_key'] = county_ids['county_key'].str.replace(pattern, '')

county_ids['county_key'] = county_ids['county_key'] + ' ' + county_ids['state_abbv']

county_ids[county_ids['state_abbv'] == 'RI']

,id,state,county,county_key,state_abbv
2310,44001,Rhode Island,Bristol County,bristol RI,RI
2311,44003,Rhode Island,Kent County,kent RI,RI
2312,44005,Rhode Island,Newport County,newport RI,RI
2313,44007,Rhode Island,Providence County,providence RI,RI
2314,44009,Rhode Island,Washington County,washington RI,RI


In [101]:
# Drop unnecessary columns for merge
county_ids.drop(['state', 'county', 'state_abbv'], axis=1, inplace=True)
county_ids.head()

,id,county_key
0,1001,autauga AL
1,1003,baldwin AL
2,1005,barbour AL
3,1007,bibb AL
4,1009,blount AL


## Merge Grant and County ID DataFrames

In [103]:
all_data = pd.merge(all_counties, county_ids, how='left', on='county_key')

all_data.head()

,state,county_orig,num_grants,grant_value,county_key,id
0,AK,aleutians east,5,123552.58,aleutians east AK,2013.0
1,AK,aleutians west,1,52405.00,aleutians west AK,2016.0
2,AK,anchorage,67,9475506.86,anchorage AK,2020.0
3,AK,bethel,3,366133.32,bethel AK,2050.0
4,AK,dillingham,2,96119.80,dillingham AK,2070.0


In [105]:
# Check for null values in id column
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2964 entries, 0 to 2963
Data columns (total 6 columns):
state          2964 non-null object
county_orig    2964 non-null object
num_grants     2964 non-null int64
grant_value    2964 non-null float64
county_key     2964 non-null object
id             2931 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 162.1+ KB


In [106]:
# Analyze the NaN values
all_data[all_data['id'].isnull()]

,state,county_orig,num_grants,grant_value,county_key,id
9,AK,kenai-cook inlet,1,20000.00,kenai-cook inlet AK,NaN
17,AK,prince of wales hyder,1,98280.00,prince of wales hyder AK,NaN
18,AK,prince wales ketchikan,7,289044.71,prince wales ketchikan AK,NaN
23,AK,valdez-chitina-w,1,29138.20,valdez-chitina-w AK,NaN
25,AK,wrangell petersburg,4,604662.86,wrangell petersburg AK,NaN
51,AL,de kalb,32,1910939.96,de kalb AL,NaN
321,FL,dade,47,18499632.73,dade FL,NaN
331,FL,hills,2,177350.00,hills FL,NaN
403,GA,de kalb,1,8204.80,de kalb GA,NaN
536,IA,dorado,1,49578.00,dorado IA,NaN
